In [ ]:
import numpy as np

# исходная функция, которую нужно аппроксимировать моделью a(x)
def func(x):
    return 0.5 * x**2 - 0.1 * 1/np.exp(-x) + 0.5 * np.cos(2*x) - 2.

coord_x = np.arange(-5.0, 5.0, 0.1) # значения по оси абсцисс [-5; 5] с шагом 0.1
coord_y = func(coord_x) # значения функции по оси ординат

sz = len(coord_x)	# количество значений функций (точек)
eta = np.array([0.01, 0.001, 0.0001, 0.01, 0.01]) # шаг обучения для каждого параметра w0, w1, w2, w3, w4
w = np.array([0., 0., 0., 0., 0.]) # начальные значения параметров модели
N = 500 # число итераций алгоритма SGD
lm = 0.02 # значение параметра лямбда для вычисления скользящего экспоненциального среднего
x_i = np.column_stack((np.ones(sz), coord_x, coord_x ** 2, np.cos(2*coord_x), np.sin(2*coord_x)))                         
Qe = np.mean((x_i @ w.T - coord_y) ** 2) # начальное значение среднего эмпирического риска
np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел
L = np.array([])

for i in range(N):
    k = np.random.randint(0, sz-1)

    x = x_i[k]

    l = (x @ w.T - coord_y[k]) ** 2
    L = np.append(L,l)

    dl = 2 * (x @ w.T - coord_y[k]) * x.T

    w -= eta * dl

    Qe = lm * l + (1-lm) * Qe

Q = np.mean((x_i @ w.T - coord_y) ** 2) 

In [ ]:
import numpy as np

# исходная функция, которую нужно аппроксимировать моделью a(x)
def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.05 * x ** 3 + 0.2 * np.sin(4 * x) - 2.5


coord_x = np.arange(-4.0, 6.0, 0.1) # значения по оси абсцисс [-4; 6] с шагом 0.1
coord_y = func(coord_x) # значения функции по оси ординат

sz = len(coord_x)	# количество значений функций (точек)
eta = np.array([0.1, 0.01, 0.001, 0.0001]) # шаг обучения для каждого параметра w0, w1, w2, w3
w = np.array([0., 0., 0., 0.]) # начальные значения параметров модели
N = 500 # число итераций алгоритма SGD
lm = 0.02 # значение параметра лямбда для вычисления скользящего экспоненциального среднего
batch_size = 50 # размер мини-батча (величина K = 50)
x_i = np.column_stack((np.ones(sz), coord_x, coord_x ** 2, coord_x ** 3))

Qe = np.mean((x_i @ w.T - coord_y) ** 2)
np.random.seed(0) # генерация одинаковых последовательностей псевдослучайных чисел

for _ in range(N):
    k = np.random.randint(0, sz-batch_size-1)
    x_batch = x_i[k:k+batch_size]
    y_batch = coord_y[k:k+batch_size]

    y_pred = x_batch @ w.T

    error = y_pred - y_batch
    Qk = np.mean(error ** 2)  # усечённый эмпирический риск для мини-батча

    # Обновление скользящего экспоненциального среднего эмпирического риска
    Qe = lm * Qk + (1 - lm) * Qe

    er_col = error.reshape(batch_size, 1)
    # Вычисление псевдоградиента
    grad = 2 * np.mean(er_col * x_batch, axis=0)
    
    # Обновление параметров модели
    w -= eta * grad


Q = np.mean((x_i @ w.T - coord_y) ** 2)